Part a

In [528]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

# 加载数据文件，并对日期进行预处理
# Load the data file and preprocess the dates.
nikkei_data = pd.read_csv('./hw2 data/nikkei_daily.txt', sep='\t', parse_dates=['Date'])
sp_data = pd.read_csv('./hw2 data/sp_daily.txt', sep=r'\s+')
ex_data = pd.read_csv('./hw2 data/ex_daily.txt', sep=r'\s+')

nikkei_data.set_index('Date', inplace=True)
sp_data['caldt'] = pd.to_datetime(sp_data['caldt'].astype(str), format='%Y%m%d')
sp_data.set_index('caldt', inplace=True)
sp_data.index.name = 'Date'
ex_data['DATE'] = pd.to_datetime(ex_data['DATE'])
ex_data.set_index('DATE', inplace=True)
ex_data.index.name = 'Date'

# 根据题目所给区间过滤指定日期范围的数据
# Filter the data within the specified date range according to the interval given in the question
start_date = '2005-01-03'
end_date = '2007-12-31'

# 获取指定日期范围内所有三个数据集都有的日期
# Obtain the dates that exist in all three datasets within the specified date range.
nikkei_dates = set(nikkei_data.loc[start_date:end_date].index)
sp_dates = set(sp_data.loc[start_date:end_date].index)
ex_dates = set(ex_data.loc[start_date:end_date].index)
common_dates = sorted(nikkei_dates & sp_dates & ex_dates)

# 创建一个包含共同日期的数据框
# Create a dataframe containing the common dates
result = pd.DataFrame(index=common_dates)
result.index.name = 'Date'

# 对每个日期，检查其是否有可用的(i-1)天数据和(i-2)天数据，以筛选出可用数据点
# For each date, check if there is available data for the (i - 1)th day and the (i - 2)th day to filter out the available data points.
for i in range(len(common_dates)):
    current_date = common_dates[i]

    prev_date = common_dates[i-1]
    prev_prev_date = common_dates[i-2]
    
    days_since_prev = (current_date - prev_date).days
    days_between_prev = (prev_date - prev_prev_date).days

    if days_since_prev == 1 and days_between_prev == 1:

        # g
        nikkei_current = nikkei_data.loc[current_date, 'Value']
        nikkei_prev = nikkei_data.loc[prev_date, 'Value']
        result.loc[current_date, 'g'] = 1 if nikkei_current > nikkei_prev else -1
        
        # x1
        result.loc[current_date, 'x1'] = sp_data.loc[current_date, 'sprtrn']
        
        # x2
        ex_prev = ex_data.loc[prev_date, 'VALUE']
        ex_prev_prev = ex_data.loc[prev_prev_date, 'VALUE']
        result.loc[current_date, 'x2'] = np.log(ex_prev) - np.log(ex_prev_prev)

result = result.dropna()

# 打印经过筛查后得到的可用数据的前几项，经过人工检验确认无误
# Print the first few items of the available data obtained after screening. Manual inspection confirms that they are correct.
print(result.head())

# 提取特征和标签
# Extract features and labels
X = result[['x1', 'x2']].values
label = result['g'].values

# 按题设划分后60天作为测试集，其余样本为训练集，以预备性能评估。
# Divide the data according to the problem statement, with the last 60 days as the test set and the remaining samples as the training set, in preparation for performance evaluation.
# 同时，为了优化分类器性能，这里使用多项式特征工程添加了特征：x1^2, x2^2, x1*x2. 故输入的特征向量维数为5。
# Meanwhile, to optimize the performance of the classifier, polynomial feature engineering is used here to add features: x1^2, x2^2, x1*x2. Therefore, the dimension of the input feature vector is 5.
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)

X_train_poly = X_poly[:-60]
X_test_poly = X_poly[-60:]

# 最后，对需要使用的输入特征进行标准化。
# Finally, standardize the input features that need to be used.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_poly) 
X_test = scaler.transform(X_test_poly)

# X_train = scaler.fit_transform(X[:-60]) 
# X_test = scaler.transform(X[-60:])

label_train = label[:-60]
label_test = label[-60:]

              g        x1        x2
Date                               
2005-01-06  1.0  0.003506 -0.003074
2005-01-07 -1.0 -0.001431  0.008811
2005-01-13 -1.0 -0.008630 -0.011280
2005-01-14  1.0  0.006005  0.002832
2005-01-20 -1.0 -0.007783  0.001562


Part b

In [529]:
# (i).Logistic regression

logistic = LogisticRegression(random_state=0)
logistic.fit(X_train, label_train)

# 计算训练误差率
# Calculate the training error rate
logistic_hat1 = logistic.predict(X_train)
logistic_ac1 = accuracy_score(label_train, logistic_hat1)
logistic_er1 = 1 - logistic_ac1

# 计算测试误差率
# Calculate the test error rate
logistic_hat2 = logistic.predict(X_test)
logistic_ac2 = accuracy_score(label_test, logistic_hat2)
logistic_er2 = 1 - logistic_ac2

print('The error rate of logistic regression on the training set is:', f'{logistic_er1:.4f}')
print('The error rate of logistic regression on the test set is:', f'{logistic_er2:.4f}')

The error rate of logistic regression on the training set is: 0.4307
The error rate of logistic regression on the test set is: 0.4333


In [530]:
def kfold(X, y, n=10, test_size=None, gap=0):
    """
    CV的准备函数，用于根据折数分割训练集和验证集
    The preparation function for CV, which is used to split the training set and validation set according to the number of folds
    """
    n_samples = len(X)
    if test_size is None:
        test_size = n_samples // (n + 1)    
    
    # 处理余数。由于特征的时间序列特性，同时确保验证集索引不在训练集之前
    # Handle the remainder. Due to the time - series characteristics of the features, also ensure that the validation set indices are not before those of the training set.
    folds = []
    train_size = n_samples - n * test_size
    for i in range(n):
        train_end = train_size + i * test_size
        test_start = train_end + gap
        test_end = test_start + test_size
        if test_end > n_samples:
            test_end = n_samples
            if test_start >= test_end:
                break
        
        train_indices = np.arange(train_end)
        test_indices = np.arange(test_start, test_end)
        folds.append((train_indices, test_indices))
    return folds


def cross_validation_1(X, y, C_range, n=10):
    """
    线性核SVM的交叉验证函数，默认10折，寻找最优的参数C
    The cross-validation function for linear-kernel SVM, with the default of 10-fold, to find the optimal parameter C
    """
    folds = kfold(X, y, n=n)
    best_score = -np.inf
    best_params = None
    
    # 对每个c进行循环
    # Loop through each c
    for c in C_range:
        scores = []
        
        # 对每一折进行训练和验证
        # Train and validate for each fold
        for train_idx, val_idx in folds:
            X_train_fold, y_train_fold = X[train_idx], y[train_idx]
            X_val_fold, y_val_fold = X[val_idx], y[val_idx]
            
            # 使用当前参数创建并训练模型
            # Create and train the model with the current parameters
            model = SVC(C=c, kernel='linear', random_state=0)
            model.fit(X_train_fold, y_train_fold)
            
            # 在验证集上计算准确率
            # Calculate the accuracy on the validation set
            y_pred = model.predict(X_val_fold)
            accuracy = np.mean(y_pred == y_val_fold)
            scores.append(accuracy)
        
        # 计算平均准确率，并更新最佳参数
        # Calculate the average accuracy, and update the best parameters
        mean_score = np.mean(scores)
        if mean_score > best_score:
            best_score = mean_score
            best_params = c
    
    return best_params

In [531]:
# (ii).SVM with linear kernel

# 设定参数C的范围并进行10折交叉验证找到最优的C
# Set the range of parameter C and perform 10-fold cross-validation to find the optimal C.
C_range = [0.1, 1, 5, 10, 100]
best_c = cross_validation_1(X_train, label_train, C_range, n=10)

# 使用最优的参数C训练svm_linear模型
# Train the svm_linear model using the optimal parameter C
svm_linear = SVC(C=best_c, kernel='linear', random_state=0)
svm_linear.fit(X_train, label_train)

# 计算训练误差率
# Calculate the training error rate
svm_linear_hat1 = svm_linear.predict(X_train)
svm_linear_ac1 = accuracy_score(svm_linear_hat1, label_train)
svm_linear_er1 = 1 - svm_linear_ac1

# 计算测试误差率
# Calculate the test error rate
svm_linear_hat2 = svm_linear.predict(X_test)
svm_linear_ac2 = accuracy_score(svm_linear_hat2, label_test)
svm_linear_er2 = 1 - svm_linear_ac2

print('The error rate of the linear-kernel SVM on the training set is:', f'{svm_linear_er1:.4f}')
print('The error rate of the linear-kernel SVM on the test set is:', f'{svm_linear_er2:.4f}')

The error rate of the linear-kernel SVM on the training set is: 0.4428
The error rate of the linear-kernel SVM on the test set is: 0.5000


In [532]:
def cross_validation_2(X, y, parameters_range, n=10):
    """
    线性核SVM的交叉验证函数，默认10折，寻找最优的参数C和gamma(σ)
    The cross-validation function for radial-kernel SVM, with the default of 10-fold, to find the optimal parameter C and gamma(σ)
    因为大部分都和cross_validation_1函数一样，所以我就不写详细的注释了~
    Since most of it is the same as the cross_validation_1 function, detailed comments won't be written.
    """
    folds = kfold(X, y, n=n)
    best_score = -np.inf
    best_params = None
    
    for c in parameters_range['C']:
        for gamma in parameters_range['gamma']:
            scores = []
            
            for train_idx, val_idx in folds:
                X_train_fold, y_train_fold = X[train_idx], y[train_idx]
                X_val_fold, y_val_fold = X[val_idx], y[val_idx]

                model = SVC(C=c, kernel='rbf', gamma=gamma, random_state=0)
                model.fit(X_train_fold, y_train_fold)
                
                y_pred = model.predict(X_val_fold)
                accuracy = np.mean(y_pred == y_val_fold)
                scores.append(accuracy)
            
            mean_score = np.mean(scores)
            if mean_score > best_score:
                best_score = mean_score
                best_params = {'C': c, 'gamma': gamma}
    
    return best_params

In [533]:
# (iii).SVM with radial kernel

# 用字典存储参数C和gamma（有γ = 1/(2 * σ^2)）的设定范围，并使用10折交叉验证得到最优的c和gamma
# Store the set ranges of parameters C and gamma (where γ = 1/(2 * σ²)) in a dictionary,and use 10-fold cross-validation to obtain the optimal c and gamma.
parameters_range = {'C': [5, 10, 100], 
                  'gamma': [5, 10]}
best_params = cross_validation_2(X_train, label_train, parameters_range, n=10)
best_c = best_params['C']
best_gamma = best_params['gamma']
best_sigma = np.sqrt(1/(2*best_gamma)) if best_gamma > 0 else float('inf')

svm_rbf = SVC(C=best_c, kernel='rbf', gamma=best_gamma, random_state=0)
svm_rbf.fit(X_train, label_train)

# 计算训练误差率
# Calculate the training error rate
svm_rbf_hat1 = svm_rbf.predict(X_train)
svm_rbf_ac1 = accuracy_score(svm_rbf_hat1, label_train)
svm_rbf_er1 = 1 - svm_rbf_ac1

# 计算测试误差率
# Calculate the test error rate
svm_rbf_hat2 = svm_rbf.predict(X_test)
svm_rbf_ac2 = accuracy_score(svm_rbf_hat2, label_test)
svm_rbf_er2 = 1 - svm_rbf_ac2

print('The error rate of the radial-kernel SVM on the training set is:', f'{svm_rbf_er1:.4f}')
print('The error rate of the radial-kernel SVM on the test set is:', f'{svm_rbf_er2:.4f}')

The error rate of the radial-kernel SVM on the training set is: 0.1446
The error rate of the radial-kernel SVM on the test set is: 0.4333


In [534]:
# 总结比较
# Summary and comparison
print('\n\t===== Comparison of Error Rates of Three Models =====')
print('Model\t\t\tTraining Set Error Rate\t\tTest Set Error Rate')
print(f'Logistic Regression\t{logistic_er1:.4f}\t\t\t\t{logistic_er2:.4f}')
print(f'Linear-kernel SVM\t{svm_linear_er1:.4f}\t\t\t\t{svm_linear_er2:.4f}')
print(f'Radial-kernel SVM\t{svm_rbf_er1:.4f}\t\t\t\t{svm_rbf_er2:.4f}')


	===== Comparison of Error Rates of Three Models =====
Model			Training Set Error Rate		Test Set Error Rate
Logistic Regression	0.4307				0.4333
Linear-kernel SVM	0.4428				0.5000
Radial-kernel SVM	0.1446				0.4333


虽然目前看来训练的三个模型正确率都不十分高，不过我在给的所有数据上试验发现：当样本量变大后，三个模型都能显著优于随机分类，SVM分类的正确率更是可以超过0.6。说明应该还是有一定功能的。